In [355]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [356]:
df = pd.DataFrame(pd.read_csv('scraped_and_full_data.csv'))

In [357]:
df.columns

Index(['Unnamed: 0', 'date', 'companyId', 'jobId', 'country', 'stateProvince',
       'city', 'avgOverallRating', 'numReviews', 'industry', 'normTitle',
       'normTitleCategory', 'descriptionCharacterLength',
       'descriptionWordCount', 'experienceRequired', 'estimatedSalary',
       'salaryCurrency', 'jobLanguage', 'supervisingJob', 'licenseRequiredJob',
       'educationRequirements', 'jobAgeDays', 'clicks', 'localClicks', 'State',
       'Number of Residents in Millions', '2017rate', 'Index', 'Grocery',
       'Housing', 'Utilities', 'Transportation', 'Heath', 'Misc.'],
      dtype='object')

Subset to only needed columns and subset NA values.

In [383]:
unneeded = ['Unnamed: 0', 'date', 'companyId', 'country', 'avgOverallRating', 'numReviews', 'descriptionCharacterLength',
       'descriptionWordCount', 'experienceRequired','salaryCurrency', 'jobLanguage', 'supervisingJob', 'licenseRequiredJob',
       'educationRequirements', 'jobAgeDays', 'clicks', 'localClicks', '2017rate', 
       'clickRatio', "industry"]

In [384]:
df_important = df.drop([col for col in unneeded if col in df], axis = 1)

In [361]:
len(df_important)

11388500

In [362]:
df_important = df_important.dropna(subset=["normTitleCategory"])
df_important.head()

,jobId,stateProvince,city,normTitle,normTitleCategory,estimatedSalary,State,Number of Residents in Millions,Index,Grocery,Housing,Utilities,Transportation,Heath,Misc.
0,job0000002,AZ,Peoria,customer service representative,customer,22800,Arizona,7.02,95.6,100.1,89.5,98.3,99.6,98.1,96.7
1,job0000416,AZ,Dewey-Humboldt,medical assistant,medtech,27000,Arizona,7.02,95.6,100.1,89.5,98.3,99.6,98.1,96.7
2,job0000420,AZ,Scottsdale,executive assistant to the president,admin,43700,Arizona,7.02,95.6,100.1,89.5,98.3,99.6,98.1,96.7
3,job0000420,AZ,Scottsdale,executive assistant to the president,admin,43700,Arizona,7.02,95.6,100.1,89.5,98.3,99.6,98.1,96.7
4,job0000420,AZ,Scottsdale,executive assistant to the president,admin,43700,Arizona,7.02,95.6,100.1,89.5,98.3,99.6,98.1,96.7


In [363]:
(df_important.drop_duplicates()["jobId"].value_counts() == 1).value_counts()

True     371732
False     16358
Name: jobId, dtype: int64

In [364]:
df_important = df_important.drop_duplicates()
subset = df_important["jobId"].value_counts() == 1
subset = list(subset[subset == True].keys())
df_all_listings = df_important.loc[df_important["jobId"].isin(subset)]

In [366]:
len(df_all_listings)
#Dealing with 371,732 individual job listings

371732

We will group by stateProvince to get the average cost of living index for each state.

In [367]:
state_cost_index = pd.DataFrame(df_all_listings.groupby(["stateProvince"])["Index"].mean().reset_index())
state_cost_index

,stateProvince,Index
0,AK,131.3
1,AL,90.3
2,AR,87.8
3,AZ,95.6
4,CA,141.0
5,CO,102.3
6,CT,125.7
7,DE,102.9
8,FL,99.3
9,GA,90.8


In [368]:
job_state_avgsal= pd.DataFrame(df_all_listings.groupby(["stateProvince", "normTitleCategory"])["estimatedSalary"].mean()).reset_index()
job_counts_typestate = pd.DataFrame(None)
job_counts_typestate["Job Category Frequency"]= df_all_listings.groupby(["stateProvince", "normTitleCategory"])["normTitleCategory"].count()
job_counts_typestate = job_counts_typestate.reset_index()


In [373]:
###Here df_sub_total is the final product dataset we will need 
df_sub_total = job_state_avgsal.merge(job_counts_typestate)
df_sub_total = df_sub_total.merge(state_cost_index, on = "stateProvince")
df_sub_total["estimatedSalary/Index"] = df_sub_total["estimatedSalary"]/df_sub_total["Index"]
df_sub_total

,stateProvince,normTitleCategory,estimatedSalary,Job Category Frequency,Index,estimatedSalary/Index
0,AK,accounting,67490.000000,10,131.3,514.013709
1,AK,admin,34445.238095,42,131.3,262.339970
2,AK,arts,60933.333333,3,131.3,464.077177
3,AK,aviation,58400.000000,5,131.3,444.782940
4,AK,childcare,27492.857143,14,131.3,209.389620
5,AK,construction,41337.500000,8,131.3,314.832445
6,AK,customer,26790.909091,33,131.3,204.043481
7,AK,driver,36328.571429,14,131.3,276.683712
8,AK,education,44733.333333,15,131.3,340.695608
9,AK,engchem,90700.000000,1,131.3,690.784463


The top ratio value for every state was for medical doctors (meaning it is most sustainable to be a doctor in any state), but not all doctors have at least 10 entries in each state. Also for the purpose of true inference of more sustainable situations in each state, we chose to exclude medical doctors from the analysis and look at the top 3 greatest salary/costOfLivingIndex ratios. 

In [393]:
df_branch = df_sub_total[df_sub_total["normTitleCategory"] != "meddr"]
df_branch = df_branch[df_branch["Job Category Frequency"] >= 10]

In [395]:
max_ratio = df_branch.groupby(["stateProvince"])["estimatedSalary/Index"].nlargest(3).reset_index()
df_best_ratio_by_state = df_branch.merge(max_ratio)
df_best_ratio_by_state 

,stateProvince,normTitleCategory,estimatedSalary,Job Category Frequency,Index,estimatedSalary/Index,level_1
0,AK,accounting,67490.000000,10,131.3,514.013709,0
1,AK,mednurse,67868.518519,108,131.3,516.896561,24
2,AK,therapy,72500.000000,20,131.3,552.170602,42
3,AL,engcivil,84842.105263,19,90.3,939.558198,59
4,AL,engid,81550.000000,32,90.3,903.100775,61
5,AL,techsoftware,86101.204819,83,90.3,953.501714,96
6,AR,engid,80318.181818,11,87.8,914.785670,115
7,AR,techinfo,79990.909091,22,87.8,911.058190,147
8,AR,techsoftware,86741.860465,43,87.8,987.948297,148
9,AZ,engid,88557.407407,54,95.6,926.332713,169


In [370]:
#here are the average estimated salaries across states
df_sub_total.groupby(["normTitleCategory"])["estimatedSalary"].mean().sort_values(ascending = False)

normTitleCategory
meddr            146420.562942
techsoftware      89591.707984
engid             79828.372474
engmech           77271.812512
project           76976.175286
engcivil          75072.702529
engelectric       73459.449902
arch              72796.732755
engchem           72138.726330
techinfo          70342.875791
math              70016.535641
socialscience     65144.907925
legal             63859.646301
therapy           63792.779956
techhelp          63773.148390
mednurse          58330.543536
finance           56900.711950
aviation          55217.380858
accounting        54643.264059
hr                53991.581480
science           53801.780788
meddental         53298.364104
marketing         50550.755707
management        50166.289266
sales             47845.572643
insurance         47748.903444
arts              47668.505384
pharmacy          47044.262253
military          46991.285181
media             46754.543624
realestate        46588.807080
tech              442

In [371]:
#Here are the total number of jobs being listed in each state in our data
df_sub_total.groupby(["stateProvince"])["Job Category Frequency"].sum().sort_values(ascending = False)


stateProvince
CA    43107
TX    30291
FL    21577
NY    20378
PA    16450
IL    15994
OH    14502
VA    12462
GA    12136
NC    11996
MI    11004
MA    10657
NJ    10149
WA     9066
CO     8635
IN     8618
MN     8509
AZ     8149
MD     8107
WI     7581
TN     7517
MO     7119
SC     5649
OR     4734
KY     4533
AL     4172
CT     3994
IA     3832
OK     3709
UT     3545
LA     3436
KS     3408
NV     2972
AR     2620
NE     2200
NM     2091
NH     2017
MS     1948
WV     1606
ME     1537
ID     1392
DE     1282
RI     1163
HI     1114
SD      962
MT      945
ND      889
VT      758
AK      671
WY      549
Name: Job Category Frequency, dtype: int64